<a href="https://colab.research.google.com/github/hhubert14/chess-ai/blob/main/chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
# !pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers==4.38.0
!pip install --upgrade transformers
!pip install --upgrade peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `safafa` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `safafa`


In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Ensures synchronous error reporting
os.environ['TORCH_USE_CUDA_DSA'] = "1"    # Enables device-side assertions

# !SET TORCH_USE_CUDA_DSA = 1

In [8]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [7]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate.test_utils.testing import get_backend
from tqdm.auto import tqdm
from peft import LoftQConfig, LoraConfig, get_peft_model
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

import torch

# TODO add validation set

# Adjustable variables
model_name = "google/gemma-2-2b"
batch_size = 4
train_dataset_path = "/content/train_dataset.csv"
test_dataset_path = "/content/test_dataset.csv"
num_epochs = 5
learning_rate = 5e-5
model_dir = f"{model_name}_saved"

# "google/flan-t5-base"

# model_config = {"rope_scaling": {"type": "linear", "factor": 8.0}}

tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, config=model_config)
model = AutoModelForCausalLM.from_pretrained(model_name)
# model.gradient_checkpointing_enable()
# model.half()

peft_config = LoraConfig(inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, peft_type=TaskType.CAUSAL_LM)
# loftq_config = LoftQConfig(loftq_bits=4, ...)           # set 4bit quantization
# lora_config = LoraConfig(..., init_lora_weights="loftq", loftq_config=loftq_config)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())


# Load datasets in streaming mode
train_dataset = load_dataset("csv", data_files={"full": train_dataset_path}, streaming=True)["full"]
eval_dataset = load_dataset("csv", data_files={"full": test_dataset_path}, streaming=True)["full"]

# Tokenize dynamically using a collate function
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token_id = tokenizer.pad_token_id  # Ensure tokenizer uses the same pad token for labels
tokenizer.label_pad_token_id = tokenizer.pad_token_id  # Set

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,597,440 || all params: 2,615,939,328 || trainable%: 0.0611
None


In [49]:
def tokenize_batch(batch):
    inputs_text = [example["inputs"] for example in batch]
    labels_text = [example["label"] for example in batch]

    # print(f"Number of inputs: {len(inputs_text)}, Number of labels: {len(labels_text)}")
    # assert len(inputs_text) == len(labels_text), "Mismatch in input and label sizes"


    # combined_text = inputs_text + labels_text
    # tokenized_batch = tokenizer(
    #     combined_text,
    #     truncation=True,
    #     padding=True,
    #     # max_length=512,
    #     return_tensors="pt"
    #     )
    # print(f"tkid: {tokenized_batch}")
    # print(f"tkid: {tokenized_batch['input_ids']}")

    # inputs = tokenizer(inputs_text, truncation=True, padding=True, max_length=512, return_tensors="pt")
    # labels = tokenizer(labels_text, truncation=True, padding=True, max_length=512, return_tensors="pt")
    inputs = tokenizer(
        inputs_text,
        truncation="only_first",
        padding="max_length",
        max_length=256,
        return_tensors="pt",
        add_special_tokens=True,
        )
    labels = tokenizer(
        labels_text,
        truncation="only_first",
        padding="max_length",
        max_length=256,
        return_tensors="pt",
        add_special_tokens=True,
        )

    # Replace pad token in labels with -100 to ignore during loss computation
    # labels["input_ids"][labels["input_ids"] == tokenizer.pad_token_id] = -100
    inputs["labels"] = labels["input_ids"]

    # print(f"inputs: {inputs}")
    # print(f"labels: {labels}")
    # print(f"Inputs shape: {inputs['input_ids'].shape}")
    # print(f"Labels shape: {labels['input_ids'].shape}")
    return inputs

In [50]:
# DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=tokenize_batch)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=tokenize_batch)

In [51]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Scheduler (num_training_steps calculated dynamically)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=1  # Placeholder
)

# Device setup
device, _, _ = get_backend()
print(f"Using device: {device}")
model.to(device)

Using device: cuda


PeftModel(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256001, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [52]:
# Train
progress_bar = tqdm(total=None)  # Dynamic progress bar
model.train()
step_count = 0  # Manually count steps

for epoch in range(num_epochs):
    print(f"Epoch: {epoch + 1}/{num_epochs}")
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

        step_count += 1  # Increment step count

progress_bar.close()

# Update lr_scheduler with actual training steps
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=step_count
)

# Evaluate
model.eval()
predictions_text = []
for batch in eval_dataloader:
    print("Input shape:", batch["input_ids"].shape)
    print("Labels shape:", batch["labels"].shape)
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    # Use model.generate for predicti   ons
    generated_ids = model.generate(input_ids=batch["input_ids"], max_length=512)
    # generated_ids = model.generate(input_ids=batch["input_ids"])

    decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    predictions_text.extend(decoded_preds)

print(len(predictions_text))
print(predictions_text)

model.save_pretrained(model_dir)  # Save model
tokenizer.save_pretrained(model_dir)  # Save tokenizer

print(f"Model and tokenizer saved to {model_dir}")


0it [00:00, ?it/s]

Epoch: 1/10
Epoch: 2/10
Epoch: 3/10
Epoch: 4/10
Epoch: 5/10
Epoch: 6/10
Epoch: 7/10
Epoch: 8/10
Epoch: 9/10
Epoch: 10/10


The 'batch_size' argument of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
Input shape: torch.Size([1, 256])
Labels shape: torch.Size([1, 256])
10
["It is White's turn to move. The board position is:\nr . b q k b n r\np p . . p p p p\n. . n p . . . .\n. B p . . . . .\n. . . . P . . .\n. . . . . N . .\nP P P P . P P P\nR N B Q K . . R\nThe best move is e1g1. Explain why e1g1 is the best move.\nThe best move is e1g1. Explain why e1g1 is the best move.\

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Model and tokenizer saved to google/gemma-2-2b_saved
